In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from mltools import plot_data, plot_frontiere, make_grid, gen_arti
from loss import MSELoss,CELoss,BCE
from module import Linear,Sequentiel,Optim,SGD
from fonction_activation import TanH,Sigmoide
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import seaborn as sns

In [ ]:
import scipy.io

yaleFaces = scipy.io.loadmat('./YaleFaces.mat')

X_train = yaleFaces['X_train']  # (4096, 90)
X_test = yaleFaces['X_test']    # (4096, 30)

print(X_train[0])
X_train = X_train.T
X_test = X_test.T

# Normaliser pour avoir de svaleurs entre 0 et 1
X_train = X_train / 255.0
X_test = X_test / 255.0


In [ ]:
def deflat(V):
    """" Vector[4096] -> Array[64,64] """
    return np.reshape(V, (64, 64)).T

In [ ]:
plt.imshow(deflat(X_train[0]), cmap='gray')

In [ ]:
net = Sequentiel(BCE(),Linear(4096, 2048),TanH(),Linear(2048, 1024),TanH(),Linear(1024, 2048),TanH(),Linear(2048,4096),Sigmoide())

train_loss, test_loss,X_test,y_test = SGD(
    net=net,
    loss=BCE(),
    X=X_train,
    Y=X_train,
    batch_size=32,
    epochs=500,
    eps=0.01
)

In [ ]:
import matplotlib.pyplot as plt

n = 18  # nombre d'images à afficher
reconstructed = net.forward(X_test[:n])

fig, axes = plt.subplots(2, n, figsize=(n, 2.5))
for i in range(n):


    axes[0, i].imshow(X_test[i].reshape(64, 64).T, cmap='gray')
    axes[0, i].axis('off')
    if i == n // 2:
        axes[0, i].set_title("Originales", fontsize=12)

    axes[1, i].imshow(reconstructed[i].reshape(64, 64).T, cmap='gray')
    axes[1, i].axis('off')
    if i == n // 2:
        axes[1, i].set_title("Reconstruites", fontsize=12)

plt.tight_layout()
plt.show()


In [ ]:
# 4. Visualisation
plt.figure(figsize=(12, 5))

# Courbe d'apprentissage
plt.subplot(1, 2, 1)
plt.plot(train_loss, label='Train loss')
plt.plot(test_loss, label='Test loss', linestyle='--')
plt.title("Courbe de loss"); plt.xlabel("Epochs"); plt.legend()

## différents learning_rate 


In [ ]:
#A tester aussi
learning_rate=[0.01,0.001,0.0001]

for lr in learning_rate:
    net = Sequentiel(BCE(),Linear(4096, 2048),TanH(),Linear(2048, 1024),TanH(),Linear(1024, 2048),TanH(),Linear(2048,4096),Sigmoide())

    train_loss, test_loss,X_test,y_test = SGD(
      net=net,
      loss=BCE(),
      X=X_train,
      Y=X_train,
      batch_size=32,
      epochs=100,
      eps=lr
    )
    print(np.mean(test_loss))
    # 4. Visualisation
    plt.figure(figsize=(12, 5))


    # Courbe d'apprentissage
    plt.subplot(1, 2, 1)
    plt.plot(train_loss, label='Train loss')
    plt.plot(test_loss, label='Test loss', linestyle='--')
    plt.title("Courbe de loss"); plt.xlabel("Epochs"); plt.legend()

    n = 18  # nombre d'images à afficher
    reconstructed = net.forward(X_test[:n])

    fig, axes = plt.subplots(2, n, figsize=(n, 2.5))
    for i in range(n):


        axes[0, i].imshow(X_test[i].reshape(64, 64).T, cmap='gray')
        axes[0, i].axis('off')
        if i == n // 2:
            axes[0, i].set_title("Originales", fontsize=12)

        axes[1, i].imshow(reconstructed[i].reshape(64, 64).T, cmap='gray')
        axes[1, i].axis('off')
        if i == n // 2:
            axes[1, i].set_title("Reconstruites", fontsize=12)

    plt.tight_layout()
    plt.show()

In [ ]:
def autoencoder(X_train, X_test, hidden_dim, latent_dim, epochs=1000, batch_size=32, lr=0.01):
    net = Sequentiel(
        BCE(),
        Linear(4096, hidden_dim),
        TanH(),
        Linear(hidden_dim, latent_dim),
        TanH(),
        Linear(latent_dim, hidden_dim),
        TanH(),
        Linear(hidden_dim, 4096),
        Sigmoide()
    )

    train_loss, test_loss, _, _ = SGD(
        net=net,
        loss=BCE(),
        X=X_train,
        Y=X_train,  # autoencodeur : cible = entrée
        batch_size=batch_size,
        epochs=epochs,
        eps=lr
    )

    # Reconstructions
    X_reconstructed = net.forward(X_test)

    return train_loss, test_loss, X_reconstructed, net

## différentes époches

In [ ]:
for epochs in  [500,1000,1500]:
    train_loss, test_loss, X_recon, net = autoencoder(X_train, X_test,
                                                         hidden_dim=2048,
                                                         latent_dim=1024,
                                                         epochs=epochs)

    plt.figure(figsize=(12, 5))

    # Courbe d'apprentissage
    plt.subplot(1, 2, 1)
    plt.plot(train_loss, label='Train loss')
    plt.plot(test_loss, label='Test loss', linestyle='--')
    plt.title("Courbe de loss"); plt.xlabel("Epochs"); plt.legend()


    n = 18

    fig, axes = plt.subplots(2, n, figsize=(n, 2.5))
    for i in range(n):


        axes[0, i].imshow(X_test[i].reshape(64, 64).T, cmap='gray')
        axes[0, i].axis('off')
        if i == n // 2:
            axes[0, i].set_title("Originales", fontsize=12)

        axes[1, i].imshow(X_recon[i].reshape(64, 64).T, cmap='gray')
        axes[1, i].axis('off')
        if i == n // 2:
            axes[1, i].set_title("Reconstruites", fontsize=12)

    plt.tight_layout()
    plt.show()

## différentes dimension de l'espace latent


In [ ]:
for latent_dim in [500,1000,1500]:
    train_loss, test_loss, X_recon, net = autoencoder(X_train, X_test,
                                                 hidden_dim=2048,
                                                 latent_dim=latent_dim,
                                                 epochs=500)

    plt.figure(figsize=(12, 5))

    # Courbe d'apprentissage
    plt.subplot(1, 2, 1)
    plt.plot(train_loss, label='Train loss')
    plt.plot(test_loss, label='Test loss', linestyle='--')
    plt.title("Courbe de loss"); plt.xlabel("Epochs"); plt.legend()


    n = 18

    fig, axes = plt.subplots(2, n, figsize=(n, 2.5))
    for i in range(n):


        axes[0, i].imshow(X_test[i].reshape(64, 64).T, cmap='gray')
        axes[0, i].axis('off')
        if i == n // 2:
            axes[0, i].set_title("Originales", fontsize=12)

        axes[1, i].imshow(X_recon[i].reshape(64, 64).T, cmap='gray')
        axes[1, i].axis('off')
        if i == n // 2:
            axes[1, i].set_title("Reconstruites", fontsize=12)

    plt.tight_layout()
    plt.show()

## différentes dimensions de la couche cachée

In [ ]:

for hidden_dim in [1500,1000,500]:
    train_loss, test_loss, X_recon, net = autoencoder(X_train, X_test,
                                                             hidden_dim=hidden_dim,
                                                             latent_dim=500,
                                                            epochs=500)

    plt.figure(figsize=(12, 5))

    # Courbe d'apprentissage
    plt.subplot(1, 2, 1)
    plt.plot(train_loss, label='Train loss')
    plt.plot(test_loss, label='Test loss', linestyle='--')
    plt.title("Courbe de loss"); plt.xlabel("Epochs"); plt.legend()


    n = 18

    fig, axes = plt.subplots(2, n, figsize=(n, 2.5))
    for i in range(n):


        axes[0, i].imshow(X_test[i].reshape(64, 64).T, cmap='gray')
        axes[0, i].axis('off')
        if i == n // 2:
            axes[0, i].set_title("Originales", fontsize=12)

        axes[1, i].imshow(X_recon[i].reshape(64, 64).T, cmap='gray')
        axes[1, i].axis('off')
        if i == n // 2:
            axes[1, i].set_title("Reconstruites", fontsize=12)

    plt.tight_layout()
    plt.show()


## différentes batche_size
 

In [ ]:

for batch_size in [ 32, 64, 100]:
    train_loss, test_loss, X_recon, net = autoencoder(X_train, X_test,
                                                             hidden_dim=2048,
                                                             latent_dim=1024,
                                                             epochs=500,
                                                             batch_size=batch_size)
    plt.figure(figsize=(12, 5))

    # Courbe d'apprentissage
    plt.subplot(1, 2, 1)
    plt.plot(train_loss, label='Train loss')
    plt.plot(test_loss, label='Test loss', linestyle='--')
    plt.title("Courbe de loss"); plt.xlabel("Epochs"); plt.legend()


    n = 18

    fig, axes = plt.subplots(2, n, figsize=(n, 2.5))
    for i in range(n):


        axes[0, i].imshow(X_test[i].reshape(64, 64).T, cmap='gray')
        axes[0, i].axis('off')
        if i == n // 2:
            axes[0, i].set_title("Originales", fontsize=12)

        axes[1, i].imshow(X_recon[i].reshape(64, 64).T, cmap='gray')
        axes[1, i].axis('off')
        if i == n // 2:
            axes[1, i].set_title("Reconstruites", fontsize=12)

    plt.tight_layout()
    plt.show()

## nombre de couches 


In [ ]:
net = Sequentiel(BCE(),Linear(4096, 1024),TanH(),Linear(1024, 4096),Sigmoide())


train_loss, test_loss,X_test,y_test = SGD(
      net=net,
      loss=BCE(),
      X=X_train,
      Y=X_train,
      batch_size=32,
      epochs=300,
      eps=0.01
  )

In [ ]:


n = 18  # nombre d'images à afficher
reconstructed = net.forward(X_test[:n])

fig, axes = plt.subplots(2, n, figsize=(n, 2.5))
for i in range(n):


    axes[0, i].imshow(X_test[i].reshape(64, 64).T, cmap='gray')
    axes[0, i].axis('off')
    if i == n // 2:
        axes[0, i].set_title("Originales", fontsize=12)

    axes[1, i].imshow(reconstructed[i].reshape(64, 64).T, cmap='gray')
    axes[1, i].axis('off')
    if i == n // 2:
        axes[1, i].set_title("Reconstruites", fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
# 4. Visualisation
plt.figure(figsize=(12, 5))

# Courbe d'apprentissage
plt.subplot(1, 2, 1)
plt.plot(train_loss, label='Train loss')
plt.plot(test_loss, label='Test loss', linestyle='--')
plt.title("Courbe de loss"); plt.xlabel("Epochs"); plt.legend()

In [ ]:
net = Sequentiel(BCE(),Linear(4096, 2048),TanH(),Linear(2048, 1024),TanH(),Linear(1024, 512),TanH(),Linear(512, 1024),TanH(),Linear(1024, 2048),TanH(),Linear(2048,4096),Sigmoide())


train_loss, test_loss,X_test,y_test = SGD(
      net=net,
      loss=BCE(),
      X=X_train,
      Y=X_train,
      batch_size=32,
      epochs=300,
      eps=0.01
  )

In [ ]:

import matplotlib.pyplot as plt

n = 18  # nombre d'images à afficher
reconstructed = net.forward(X_test[:n])

fig, axes = plt.subplots(2, n, figsize=(n, 2.5))
for i in range(n):


    axes[0, i].imshow(X_test[i].reshape(64, 64).T, cmap='gray')
    axes[0, i].axis('off')
    if i == n // 2:
        axes[0, i].set_title("Originales", fontsize=12)

    axes[1, i].imshow(reconstructed[i].reshape(64, 64).T, cmap='gray')
    axes[1, i].axis('off')
    if i == n // 2:
        axes[1, i].set_title("Reconstruites", fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
# 4. Visualisation
plt.figure(figsize=(12, 5))

# Courbe d'apprentissage
plt.subplot(1, 2, 1)
plt.plot(train_loss, label='Train loss')
plt.plot(test_loss, label='Test loss', linestyle='--')
plt.title("Courbe de loss"); plt.xlabel("Epochs"); plt.legend()